In [1]:
import torch
import random
import numpy as np
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm.notebook import tqdm

# Ensure deterministic behaviour
torch.backends.cudnn.determinstic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)


# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
%%capture
!pip install wandb --upgrade

In [5]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
config = dict(
    epochs=5,
    classes=10,
    kernels=[16, 32],
    batch_size=128,
    learning_rate=0.005,
    dataset="MNIST",
    architecture="CNN"
)

In [8]:
def model_pipeline(hyperparameters):
  # tell wandb to get started
  with wandb.init(project="pytorch-demo", config=hyperparameters):
    # access al HPs through wandb.config, so logging matches execution!
    config = wandb.config

    # make the model, data, and optimization problem
    model, train_loader, test_loader, criterion, optimizer = make(config)
    print(model)

    # and use them to train the model
    train(model, train_loader, criterion, optimizer, config)


    # and test its final performance
    test(model, test_loader)

  return model

In [9]:
def make(config):
  # Make the data
  train, test = get_data(train=True), get_data(train=False)
  train_loader = make_loader(train, batch_size=config.batch_size)
  test_loader = make_loader(test, batch_size=config.batch_size)

  # Make the model
  model = ConvNet(config.kernels, config.classes).to(device)

  # Make the loss and optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

  return model, train_loader, test_loader, criterion, optimizer

In [16]:
def get_data(slice=5, train=True):
    full_dataset = torchvision.datasets.MNIST(root=".",
                                              train=train, 
                                              transform=transforms.ToTensor(),
                                              download=True)
    #  equiv to slicing with [::slice] 
    sub_dataset = torch.utils.data.Subset(
      full_dataset, indices=range(0, len(full_dataset), slice))
    
    return sub_dataset


def make_loader(dataset, batch_size):
    loader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=batch_size, 
                                         shuffle=True,
                                         pin_memory=True, num_workers=8)
    return loader

In [32]:
# Conventional and convolutional neural network

class ConvNet(nn.Module):
  def __init__(self, kernels, classes=10):
    super(ConvNet, self).__init__()

    self.layer1 = nn.Sequential(
        nn.Conv2d(1, kernels[0], kernel_size=5, stride=1, padding=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(16, kernels[1], kernel_size=5, stride=1, padding=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.fc = nn.Linear(7 * 7 * kernels[-1], classes)
  
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.reshape(out.size(0), -1)
    out = self.fc(out)
    return out

In [37]:
def train(model, loader, criterion, optimizer, config):
  # tell wandb to watch what the model gets up to: gradients, weights, and more!

  wandb.watch(model, criterion, log="all", log_freq=10)

  # Run training and trach with wandb
  total_batches = len(loader) * config.epochs
  example_ct = 0 # number of examples seen
  batch_ct = 0
  for epoch in tqdm(range(config.epochs)):
    for _, (images, labels) in enumerate(loader):
      loss = train_batch(images, labels, model, optimizer, criterion)
      example_ct += len(images)
      batch_ct += 1
      
      # Report matrics every 25th batch
      if ((batch_ct + 1) % 25) == 0:
        train_log(loss, example_ct, epoch)


def train_batch(images, labels, model, optimizer, criterion):
  images, labels = images.to(device), labels.to(device)

  # Forward pss -> 
  outputs = model(images)
  loss = criterion(outputs, labels)

  # Backward pass <-
  optimizer.zero_grad()
  loss.backward()

  # Step with optimizer
  optimizer.step()

  return loss

In [38]:
def train_log(loss, example_ct, epoch):
  loss = float(loss)

  # where the magic happens
  wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
  print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")

In [39]:
def test(model, test_loader):
  model.eval()

  # Run the model on some test examples
  with torch.no_grad():
    correct, total = 0, 0
    for images, labels in test_loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)

      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
    
    print(f"Accuracy of the model on the {total} " + 
          f"test images: {100 * correct / total}%")
    
    wandb.log({"test_accuracy": correct / total})
  
  # Save the model in the exchangeable ONNX format
  torch.onnx.export(model, images, "model.onnx")
  wandb.save("model.onnx")

In [40]:
# Build, train and analyze the model with pipeline
model = model_pipeline(config)

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/5 [00:00<?, ?it/s]

Loss after 03072 examples: 0.521
Loss after 06272 examples: 0.206
Loss after 09472 examples: 0.107
Loss after 12640 examples: 0.104
Loss after 15840 examples: 0.055
Loss after 19040 examples: 0.152
Loss after 22240 examples: 0.140
Loss after 25408 examples: 0.018
Loss after 28608 examples: 0.105
Loss after 31808 examples: 0.049
Loss after 35008 examples: 0.046
Loss after 38176 examples: 0.036
Loss after 41376 examples: 0.146
Loss after 44576 examples: 0.026
Loss after 47776 examples: 0.058
Loss after 50944 examples: 0.040
Loss after 54144 examples: 0.035
Loss after 57344 examples: 0.029
Accuracy of the model on the 2000 test images: 97.85%


_runtime,11
_timestamp,1629007240
_step,57344
epoch,4
loss,0.02866
test_accuracy,0.9785


_runtime,▁▁▂▂▃▃▃▄▄▅▅▅▅▅▅▆▇▇█
_timestamp,▁▁▂▂▃▃▃▄▄▅▅▅▅▅▅▆▇▇█
_step,▁▁▂▂▃▃▃▄▄▄▅▅▆▆▆▇▇██
epoch,▁▁▁▃▃▃▃▅▅▅▅▆▆▆▆███
loss,█▄▂▂▂▃▃▁▂▁▁▁▃▁▂▁▁▁
test_accuracy,▁
